In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

# Import the ice temperature model and relevant constants
from icetemperature.lib.numerical_model import ice_temperature
from icetemperature.lib.analytical_solutions import Robin_T, Meyer_T
from icetemperature.lib.ice_properties import conductivity, heat_capacity, rate_factor
from icetemperature.lib.constants import constants
const = constants()

In [ ]:
def numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                    weertman_vel=False,dTs=0.,dH=0.,da=0.,p=12.3,ϵ_xy=0.,
                    ntsteps=1000,tol=1e-8,dt=1.,A_xy=None):
    m = ice_temperature()

    m.adot = np.array([adot/const.spy]*ntsteps)
    m.Ts = np.array([Ts]*ntsteps)
    m.H = H

    m.dTs = dTs
    m.dH = dH
    m.da = da

    m.qgeo = qgeo
    m.p = p
    m.Udef = Udef/const.spy
    m.Uslide = Uslide/const.spy

    if weertman_vel:
        m.flags.append('weertman_vel')
    m.flags.append('temp-dependent')
    m.initial_conditions()
    m.k = conductivity(m.T.copy(),m.rho)
    m.Cp = heat_capacity(m.T.copy())
    
    if A_xy is not None:
        A_xy = rate_factor(m.T,z=m.z)
    
    m.source_terms(eps_xy=ϵ_xy,A_xy=A_xy)
    m.stencil(dt*const.spy)
    m.tol=tol
    m.run_to_steady_state(eps_xy=ϵ_xy,A_xy=A_xy)
    
    return m

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,3))

ax1 = plt.subplot(133)
ax1.tick_params(labelleft=False)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax1.set_title('Dragon Margin')

ax2 = plt.subplot(132)
ax2.tick_params(labelleft=False)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax2.set_title('Kamb Ice Stream')

ax3 = plt.subplot(131)
plt.ylabel('Height Above Bed (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax3.set_title('Siple Dome')

ax1.fill_between(np.linspace(-100,100,10),-10000,100000,color='w',alpha=0.75,zorder=1)
ax2.fill_between(np.linspace(-100,100,10),-10000,100000,color='w',alpha=0.75,zorder=1)
ax3.fill_between(np.linspace(-100,100,10),-10000,100000,color='w',alpha=0.75,zorder=1)


### Dragon ###
Ts = -26.9
H = 1000.
SMB = 66.1
adot = SMB/const.rho
qgeo = 0.0633
dTs = 1e-5
dH = 0.001
Udef = 80.
Uslide = 200.
eps_xy = 2.614e-09
print('Dragon; H:',H,'T_surface:',Ts,'Accumulation Rate:',adot)

z,TR = Robin_T(Ts,qgeo,H,adot)
z,TM1 = Meyer_T(Ts,H,adot,eps_xy/2.,T_bulk=0.)
TM1 += (max(z)-z)*917.*9.81*(-7.42e-8)
z,TM2 = Meyer_T(Ts,H,adot,eps_xy/2.)
TM2 += (max(z)-z)*917.*9.81*(-7.42e-8)
m_drag = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,weertman_vel=True,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6/const.spy,
                             ϵ_xy=eps_xy)
m_drag_visc = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,weertman_vel=True,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6/const.spy,
                             ϵ_xy=eps_xy,A_xy='viscosity')

ax1.plot(TR,z,'k',zorder=1)
ax1.fill_betweenx(z,TM1,TM2,color='maroon',zorder=2,alpha=0.1)
l2, = ax1.plot(TM1,z,'maroon',zorder=2)
ax1.plot(TM2,z,'maroon',zorder=2)
ax1.fill_betweenx(m_drag.z,m_drag.T,m_drag_visc.T,color='steelblue',alpha=0.1)
ax1.plot(m_drag.T,m_drag.z,'steelblue')
ax1.plot(m_drag_visc.T,m_drag.z,'steelblue')


### Kamb ###
Ts = -25.52
H = 949.
SMB = 60.0
adot = SMB/const.rho
qgeo = .0663
dTs = 1e-5
dH = 0.001
Udef = 50.
Uslide = 250.
print('Kamb:',H,Ts,adot)

z,T = Robin_T(Ts,qgeo,H,adot)
m_kis = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,weertman_vel=True,dTs=6.42e-6,dH=-1.59e-3,da=-0.20e-6/const.spy)

l1, = ax2.plot(T,z,'k',zorder=2)
l4, = ax2.plot(m_kis.T,m_kis.z,'steelblue')


### Siple ###
Ts = -24.95
H = 1005.
SMB = 105
adot = SMB/const.rho
qgeo = .0653
print('Siple; H:',H,'T_surface:',Ts,'Accumulation Rate:',adot)

z,T = Robin_T(Ts,qgeo,H,adot)
m_siple = numerical_model(adot,Ts,H,qgeo,0.,0.)

ax3.plot(T,z,'k',zorder=1)
ax3.plot(m_siple.T,m_siple.z,'steelblue')
ax3.legend([l1,l2,l4],['Robin (1955)','Meyer (2018)','Weertman\n(1968)'])

plt.tight_layout()